In [1]:
from sklearn import datasets
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
import pandas as pd
import pymorphy2
from tqdm import tqdm_notebook as tqdm

from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import warnings

warnings.filterwarnings('ignore')

# ДЗ

In [104]:
positive = pd.read_csv('positive.csv', sep=';', header=None)
negative = pd.read_csv('negative.csv', sep=';', header=None)

In [105]:
positive_temp = positive[[3]]
positive_temp['type'] = 'positive'

negative_temp = negative[[3]]
negative_temp['type'] = 'negative'

In [106]:
# Объединим positive и negative

data = pd.concat([positive_temp, negative_temp])
data.rename(columns={3: 'text'}, inplace=True)

In [107]:
data['new_text'] = data['text'].apply(lambda x: ' '.join(str(x).lower().split()))

In [108]:
%%time

# удалим адресатов твитов, оставим только русские буквы

data['new_text'] = data['new_text'].apply(lambda x: re.sub(r'@\S+', '', x))
data['new_text'] = data['new_text'].apply(lambda x: re.sub(r'[^а-яА-Я\- ]', '', x))

CPU times: user 1.26 s, sys: 27.7 ms, total: 1.28 s
Wall time: 1.28 s


In [109]:
%%time

# оставим только буквы и приведем все к нормальной форме

morph = pymorphy2.MorphAnalyzer()

data['new_text'] = data.new_text.apply(
    lambda text: morph.parse(text)[0].normal_form
)

CPU times: user 1min 7s, sys: 228 ms, total: 1min 7s
Wall time: 1min 7s


In [110]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS

functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO'}  # междометие, частица, союз, предлог, местоимения

In [111]:
%%time

# уберем части речи из functors_pos и числа (работает 10 минут)

data['new_text'] = data.new_text.apply(
    lambda text: ' '.join(list(filter(lambda word: (pos(word) not in functors_pos) & (word.isdigit() == False), str(text).split())))
)

CPU times: user 9min 44s, sys: 4.73 s, total: 9min 49s
Wall time: 10min


In [112]:
data['new_text'] = data['new_text'].apply(lambda x: re.sub(r'[^а-яА-Я ]', '', x).lower())

In [129]:
%%time
# Преобразуем тексты в векторы признаков

TF_IDF = TfidfVectorizer(min_df=10, max_df=0.50)
CV = CountVectorizer(min_df=10, max_df=0.50)
data_cv = CV.fit_transform(data['new_text'])
data_tf_idf = TF_IDF.fit_transform(data['new_text'])

CPU times: user 5.95 s, sys: 101 ms, total: 6.06 s
Wall time: 6.06 s


In [130]:
%%time

clusterer = KMeans(n_clusters=20)
cluster_labels = clusterer.fit_predict(data_tf_idf)

CPU times: user 2min 39s, sys: 1.05 s, total: 2min 40s
Wall time: 2min 4s


In [131]:
# merge текстов с их метками
data['label'] = cluster_labels

In [132]:
from collections import Counter

for label in set(cluster_labels):
    temp = ''.join(data['new_text'][data['label'] == label]).split()
    print('НОМЕР КЛАСТЕРА: {0}'.format(label), '\n')
    print(data[data['label'] == label].groupby('type').count(), '\n')
    print(*sorted(Counter(temp).items(), key=lambda x: x[1], reverse=True)[:15])
    print()

НОМЕР КЛАСТЕРА: 0 

          text  new_text  label
type                           
negative  6384      6384   6384
positive  7196      7196   7196 

('сейчас', 1815) ('буду', 1787) ('тут', 1566) ('мой', 1463) ('может', 1307) ('быть', 1162) ('знаю', 1020) ('там', 339) ('будет', 293) ('вообще', 246) ('вс', 232) ('потом', 199) ('смотреть', 193) ('раз', 181) ('больше', 172)

НОМЕР КЛАСТЕРА: 1 

          text  new_text  label
type                           
negative  1231      1231   1231
positive  1030      1030   1030 

('новый', 1736) ('год', 1514) ('уже', 118) ('будет', 115) ('лка', 113) ('все', 78) ('этот', 77) ('скоро', 76) ('нет', 71) ('настроения', 70) ('буду', 61) ('хочу', 55) ('мой', 52) ('новогоднего', 51) ('подарок', 48)

НОМЕР КЛАСТЕРА: 2 

           text  new_text  label
type                            
negative  62112     62112  62112
positive  71714     71714  71714 

('будет', 2258) ('там', 2054) ('вс', 1791) ('вообще', 1637) ('раз', 1447) ('больше', 1272) ('такой', 1168

## в некоторых кластерах можно выделить его основную тему: 
18 кластер: не могу уснуть  
19 кластер: надо спать - завтра в школу  
13 кластер: чувствую себя плохо  
8 кластер: люблю  
6 кластер: нет настроения